In [16]:
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder

In [2]:
cd ..

/Users/alphonsowoodbury/DS/Projects/aw_flatiron_group_projects/Module3_Project


In [17]:
df1 = pd.read_csv('data/labels.csv') #target class dataset
df2 = pd.read_csv('data/values.csv') #features dataset

### Merge

In [18]:
df = pd.merge(df1, df2, on='id', how='inner')
df

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,functional,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,functional,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,functional,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,functional,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


---
## Pre-cleaning Analysis

In [19]:
df.isna().sum()

id                           0
status_group                 0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_qu

In [20]:
(df == 0).sum()

id                           1
status_group                 0
amount_tsh               41639
date_recorded                0
funder                       0
gps_height               20438
installer                    0
longitude                 1812
latitude                     0
wpt_name                     0
num_private              58643
basin                        0
subvillage                   0
region                       0
region_code                  0
district_code               23
lga                          0
ward                         0
population               21381
public_meeting            5055
recorded_by                  0
scheme_management            0
scheme_name                  0
permit                   17492
construction_year        20709
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_qu

---
## Cleaning

In [21]:
df.dropna(subset=['funder', 'scheme_management', 'public_meeting', 'permit', 'installer'], inplace=True)

---
## Pseudo-engineering

In [22]:
df['age'] = (2020 - df.construction_year)
d = {2020:np.NaN}
df.age.replace(d,inplace=True)
nancounts = df.age.isna().sum() #to get count of NaNs being replaced
df.age = df.age.fillna(df.age.mean())

In [23]:
df.population.astype('int64')
df.age = df.age.astype('int64')

In [24]:
df.status_group.value_counts()

functional                 26720
non functional             18432
functional needs repair     3499
Name: status_group, dtype: int64

---
## Drop features

In [25]:
df = df.drop(columns=['id','source_type','payment','recorded_by','construction_year',
                      'date_recorded','quality_group','quantity_group','waterpoint_type_group',
                      'num_private','extraction_type','extraction_type_group','wpt_name','scheme_name','management_group'],axis=1)


---
## Review cleaned set

In [26]:
df.head()

,status_group,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,...,permit,extraction_type_class,management,payment_type,water_quality,quantity,source,source_class,waterpoint_type,age
0,functional,6000.0,Roman,1390,Roman,34.938093,-9.856322,Lake Nyasa,Mnyusi B,Iringa,...,False,gravity,vwc,annually,soft,enough,spring,groundwater,communal standpipe,21
2,functional,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Pangani,Majengo,Manyara,...,True,gravity,vwc,per bucket,soft,enough,dam,surface,communal standpipe multiple,11
3,non functional,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mahakamani,Mtwara,...,True,submersible,vwc,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple,34
5,functional,20.0,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Pangani,Moa/Mwereme,Tanga,...,True,submersible,vwc,per bucket,salty,enough,other,unknown,communal standpipe multiple,11
6,non functional,0.0,Dwsp,0,DWSP,33.362410,-3.766365,Internal,Ishinabulandi,Shinyanga,...,True,handpump,vwc,never pay,soft,enough,machine dbh,groundwater,hand pump,23


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48651 entries, 0 to 59399
Data columns (total 27 columns):
status_group             48651 non-null object
amount_tsh               48651 non-null float64
funder                   48651 non-null object
gps_height               48651 non-null int64
installer                48651 non-null object
longitude                48651 non-null float64
latitude                 48651 non-null float64
basin                    48651 non-null object
subvillage               48288 non-null object
region                   48651 non-null object
region_code              48651 non-null int64
district_code            48651 non-null int64
lga                      48651 non-null object
ward                     48651 non-null object
population               48651 non-null int64
public_meeting           48651 non-null object
scheme_management        48651 non-null object
permit                   48651 non-null object
extraction_type_class    48651 non-null object

In [29]:
#df.describe(include='all').T

In [30]:
df.to_csv('data/cleaned_water.csv')